In [1]:
import numpy as np
import cv2
from cv2 import aruco, cvtColor, COLOR_BGR2GRAY, LUT
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from funktionen import *
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import inv as sparse_inv
from scipy.optimize import least_squares
from scipy import stats

In [150]:

f_rpi = 3386.344186
x0_rpi = 2304.
y0_rpi = 1296.
dist_rpi = np.zeros(5)
  
kameratypen = pd.DataFrame([[0, "rpi", f_rpi, x0_rpi, y0_rpi, dist_rpi[0], dist_rpi[1], dist_rpi[2], dist_rpi[3], dist_rpi[4]]],
                           columns=["kameratyp", "name", "f", "cx", "cy", "d1","d2","d3","d4","d5"]).astype({"kameratyp": int}).set_index(["kameratyp"])
kameras = pd.DataFrame([], columns=["kamera", "kameratyp", "name", "df", "dcx", "dcy"]).astype({"kamera": int, "kameratyp": int, "df": float, "dcx":float, "dcy":float}).set_index(["kamera"])

pictures = pd.DataFrame( columns=["img", "kamera",
                        "pfad", "t1", "t2", "t3", "r1", "r2", "r3"],  dtype=float).astype({"img": int, "kamera": int, "pfad": str}).set_index(["img"])

In [4]:
pictures

,kamera,pfad,t1,t2,t3,r1,r2,r3
img,,,,,,,,


In [5]:
pictures.dtypes

kamera      int64
pfad       object
t1        float64
t2        float64
t3        float64
r1        float64
r2        float64
r3        float64
dtype: object

In [6]:
kameratypen

,name,f,cx,cy,d1,d2,d3,d4,d5
kameratyp,,,,,,,,,
0,rpi,3386.344186,2304.0,1296.0,0.0,0.0,0.0,0.0,0.0


In [151]:
pfad = glob("../../bilderserien/33a7d422-5c80-4444-826e-375efb7da027.jpg/*.jpg")

rpi_kameras = list(set([i[-12:-4] for i in pfad]))
rpi_kameras.sort()

for i in rpi_kameras:
    kamera_id = len(kameras)
    kameras.loc[kamera_id] = [0,i, 0., 0., 0.]

for i in pfad:
    kamera = kameras.index[kameras['name'] == i[-12:-4]][0]
    kamera = 0
    pictures.loc[len(pictures), ['kamera', 'pfad']] = [int(kamera), i]


In [152]:
pictures = pictures.astype({ "kamera": int, "pfad": str, "t1": float, "t2": float, "t3": float, "r1": float, "r2": float, "r3": float})
pictures

,kamera,pfad,t1,t2,t3,r1,r2,r3
img,,,,,,,,
0,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
1,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
2,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
3,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
4,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
5,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
6,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
7,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
8,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
kameras = kameras.where(kameras.reset_index()['kamera'] == 0).dropna().reset_index().astype({"kamera": int, "kameratyp": int, "df": float, "dcx":float, "dcy":float}).set_index(["kamera"])


In [186]:
kameras

,kameratyp,name,df,dcx,dcy
kamera,,,,,
0,0,camera01,0.0,0.0,0.0


In [9]:
aruco_dict = aruco.extendDictionary(32, 3)
parameter = aruco.DetectorParameters()
parameter.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
LUT_IN = [0, 158, 216, 255]
LUT_OUT = [0, 22, 80, 176]
lut = np.interp(np.arange(0, 256),
                LUT_IN, LUT_OUT).astype(np.uint8)

In [10]:
corners = []
for j, img in pictures.iterrows():
    imgCV = cv2.imread(img['pfad'])
    gray = cvtColor(imgCV, COLOR_BGR2GRAY)
    tmp_corners, tmp_ids, t = aruco.detectMarkers(
        gray, aruco_dict, parameters=parameter)
    for c, i in zip(tmp_corners, tmp_ids):
        size = np.linalg.norm(c[0][0] - c[0][1])
        # print(size)
        if size < 100:
            continue
        for k in range(len(c[0])):
            corners.append([j, i[0], k, c[0][k][0], c[0][k][1]])
corners = pd.DataFrame(corners, columns=["img", "marker", "ecke", "x", "y"])
corners.set_index(["img", "marker", "ecke"], inplace=True)


In [11]:
corners

x            y
img marker ecke                          
0   14     0      220.645828  2092.486328
           1      316.570404  2173.849121
           2      292.321411  2359.607178
           3      190.585388  2284.216309
    25     0     4081.205078  2175.026855
...                      ...          ...
23  10     3     2674.674561   729.215515
    30     0     1130.000000   855.000000
           1     1029.000000   843.000000
           2     1029.000000   670.000000
           3     1135.000000   686.000000

[560 rows x 2 columns]

In [158]:
coords = pd.DataFrame([[31, 0, -1.7, 1.7, 0.0],
                       [31, 1, 1.7, 1.7, 0.0],
                       [31, 2, 1.7, -1.7, 0.0],
                       [31, 3, -1.7, -1.7, 0.0]], columns=["marker", "ecke", "x", "y", "z"]).astype({"marker": int, "ecke": int}).set_index(["marker", "ecke"])
coords

x    y    z
marker ecke               
31     0    -1.7  1.7  0.0
       1     1.7  1.7  0.0
       2     1.7 -1.7  0.0
       3    -1.7 -1.7  0.0

In [159]:
ca = corners.reset_index().set_index(["marker", "ecke"])
matched = ca.join(ca, lsuffix="_a", rsuffix="_b").query(
    'img_a < img_b')
matched.reset_index(inplace=True)
matched

,marker,ecke,img_a,x_a,y_a,img_b,x_b,y_b
0,0,0,10,1131.854980,1195.923584,14,2544.997559,312.697754
1,0,0,10,1131.854980,1195.923584,16,4401.152832,637.118408
2,0,0,10,1131.854980,1195.923584,17,4174.938965,599.293701
3,0,0,14,2544.997559,312.697754,16,4401.152832,637.118408
4,0,0,14,2544.997559,312.697754,17,4174.938965,599.293701
...,...,...,...,...,...,...,...,...
1447,31,2,2,2268.268066,1765.975830,12,2416.334473,2047.898438
1448,31,2,11,2388.193115,2384.410400,12,2416.334473,2047.898438
1449,31,3,2,2824.286377,1977.494751,11,1617.826294,2370.907959
1450,31,3,2,2824.286377,1977.494751,12,2204.228027,1835.252808


In [19]:
def get_kamera(kameras, kid):
    kamera = kameras.iloc[kid]
    kameratyp = kameratypen.iloc[kamera["kameratyp"]]
    mtx = np.array([[kamera["df"]+ kameratyp["f"], 0, kamera["dcx"]+ kameratyp["cx"]],
                    [0, kamera["df"]+ kameratyp["f"], kamera["dcy"]+ kameratyp["cy"]],
                    [0, 0, 1]])
    dist = np.array(kameratyp[["d1", "d2", "d3", "d4", "d5"]])
    return mtx.astype(np.float64), dist.astype(np.float64)

In [27]:
def neues_bild(coords, pictures, kameras, corners):
    lp = corners.join(coords, lsuffix="_img", rsuffix="").dropna().join(pictures, lsuffix="", rsuffix="_cam")
    lp = lp[lp["r1"].isnull()]
    bild = lp.groupby("img").count().sort_values(
        "x_img", ascending=False).first_valid_index()
    passpunkte = lp.query('img == ' + str(bild)).reset_index()
    pp = passpunkte[['x_img', 'y_img', 'x', 'y', 'z']].values
    if bild is None:
        print("nichts gefunden")
        return pictures
    kid = pictures.query('img == ' + str(bild))['kamera'].values[0 ]
    k, dist = get_kamera(kameras, kid)
    
    if len(pp) >= 4:
        _, r, t, _ = cv2.solvePnPRansac(pp[:, 2:], pp[:, :2], k, dist)
        rod, _ = cv2.Rodrigues(r)
        p = pictures.query('img == ' + str(bild))
        p[["t1", "t2", "t3", "r1", "r2", "r3"]] = [t[0][0], t[1][0], t[2][0], r[0][0], r[1][0], r[2][0]]
        pictures.update(p)
    return pictures

In [183]:
pictures = neues_bild(coords, pictures, kameras, corners)
pictures

TypeError: Cannot index by location index with a non-integer key

In [174]:
def wolke_zeigen(coords, pictures):

    # pip install plotly

    import plotly.graph_objs as go

    coords_img = []
    for _, p in pictures.dropna().iterrows():
        rt = p[['r1', 'r2', 'r3']].to_numpy(dtype=np.float64)
        R = cv2.Rodrigues(rt)[0]
        R = np.linalg.inv(R)
        t = p[['t1', 't2', 't3']].to_numpy(dtype=np.float64).T
        t = -R@t
        coords_img.append(t)
    coords_img = np.array(coords_img)
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        ))
    # Create a 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(x=coords['x'], y=coords['y'], z=coords['z'], mode='markers', name='Original Points'),
                          go.Scatter3d(x=coords_img[:, 0], y=coords_img[:, 1], z=coords_img[:, 2],
                                       mode='markers', name='Kameras')], layout=layout)

    # Add labels to the plot
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
                      title='Rotation and Translation Vectors in 3D')
    fig.layout.scene.camera.projection.type = "orthographic"
    fig.show()


wolke_zeigen(coords, pictures)

In [171]:
coords.dropna()

x    y    z
marker ecke               
31     0    -1.7  1.7  0.0
       1     1.7  1.7  0.0
       2     1.7 -1.7  0.0
       3    -1.7 -1.7  0.0

In [120]:
def neue_koordinaten(coords, pictures, kameras):
    pic_a = pictures.reset_index().rename(
        columns={"img": "img_a"}).set_index("img_a")
    pic_b = pictures.reset_index().rename(
        columns={"img": "img_b"}).set_index("img_b")
    matched = ca.add_suffix('_a').join(ca.add_suffix('_b')).query(
        'img_a < img_b').join(coords.add_suffix("_ca"))
    #print(matched)
    matched = matched[matched["x_ca"].isnull()].reset_index().set_index([
        "img_a", "img_b"])
    matched = matched.join(pic_a.add_suffix("_a"))
    matched = matched.join(pic_b.add_suffix("_b"))
    #print(matched)
    matched.dropna(subset=['t1_a', 't1_b'], inplace=True)
    matched = matched.reset_index().set_index(["marker", "ecke"])

    beste = matched.groupby(["marker"]).count().sort_values(
        "img_a", ascending=False)
    beste
    
    #print(beste)

    for bester in beste.reset_index().values[:, 0]:
        new = []
        #print(bester)
        for i in range(4):
            cneu = []

            for i, v in matched.query("marker =="+str(bester)).query("ecke =="+str(i)).iterrows():
                #print(v)
                K1, dist1 = get_kamera(kameras, v['kamera_a'])
                K2, dist2 = get_kamera(kameras, v['kamera_b'])
                R1 = cv2.Rodrigues(
                    np.array(list(v[['r1_a', 'r2_a', 'r3_a']].to_numpy(dtype=np.float64))))[0]
                R2 = cv2.Rodrigues(
                    np.array(list(v[['r1_b', 'r2_b', 'r3_b']].to_numpy(dtype=np.float64))))[0]
                t1 = v[['t1_a', 't2_a', 't3_a']].to_numpy(dtype=np.float64)
                t2 = v[['t1_b', 't2_b', 't3_b']].to_numpy(dtype=np.float64)
                P1 = np.c_[R1, t1]
                P2 = np.c_[R2, t2]
                pt1 = np.array(list(v[['x_a', 'y_a']]))
                pt2 = np.array(list(v[['x_b', 'y_b']]))
                p1n = cart2hom(cv2.undistortPoints(pt1, K1, dist1)[0][0])
                p2n = cart2hom(cv2.undistortPoints(pt2, K2, dist2)[0][0])

                xy = reconstruct_one_point(p1n, p2n, P1, P2)
                xy /= xy[3]
                cneu.append(xy[0:3])
            
            if len(cneu) == 0:
                continue
            data = pd.DataFrame(cneu, columns=["x", "y", "z"])
            #print('before',data[["x", "y", "z"]].std())
            zscore = np.abs(stats.zscore(data[["x", "y", "z"]]))
            #print(zscore)
            zscore.fillna(0, inplace=True)
            # Identify outliers as students with a z-score greater than 3
            threshold = 1.5
            data = data[zscore <= threshold].dropna()
            #print('after',data[["x", "y", "z"]].std())
            if data['x'].std() > 0.5 or data['y'].std() > 0.5 or data['z'].std() > 0.5:
                # print(data)
                continue
            # print(data[(np.abs(stats.zscore(data)) < 2).all(axis=1)])
            median = data.mean().values
            new.append([bester, i[1], median[0], median[1], median[2]])
        print(new)
        if len(new) == 4:
            new = pd.DataFrame(new, columns=["marker", "ecke", "x", "y", "z"]).set_index(
                ["marker", "ecke"])
            coords = pd.concat([coords, new])
            return coords
    return coords

In [133]:
pictures[['t1', 't2', 't3', 'r1', 'r2', 'r3']] = [None, None, None,None,None,None]

In [172]:
pictures = neues_bild(coords, pictures, kameras, corners)

nichts gefunden


In [173]:
print(len(coords    ))
coords = neue_koordinaten(coords, pictures, kameras)
print(len(coords    ))

4
[[12, 0, 2.207898156972355, 17.184244686184353, 9.946682321281894], [12, 1, 0.5601616538752291, 16.8479737930046, 9.910346014004709], [12, 2, 0.3946833352662757, 16.78006762589876, 11.619911316841149], [12, 3, 2.0387690838028614, 17.110199089527544, 11.670010349165729]]
8


In [63]:
for i in range(24):
    pictures = neues_bild(coords, pictures, kameras, corners)
    coords = neue_koordinaten(coords, pictures, kameras)

/tmp/ipykernel_47512/2126133612.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[]
[]
[]


KeyboardInterrupt: 

In [124]:

#coords.drop_duplicates(inplace=True)
#coords.drop(1, inplace=True)
coords.groupby("marker").count()

,x,y,z
marker,,,
0,4,4,4
2,4,4,4
3,4,4,4
5,4,4,4
9,4,4,4
12,4,4,4
13,4,4,4
27,4,4,4
31,4,4,4


In [125]:
wolke_zeigen(coords, pictures)

### Ausrichtung

In [101]:
def ausrichten(coords, pictures):
    bodenpunkte = coords.query("marker == 0 or marker == 6 or marker == 15 or marker == 3 or marker == 11 or marker == 13 or marker == 14 or marker == 4").query(
        "ecke == 1 or ecke == 2")
    hoeherepunkte = coords.query("marker == 0 or marker == 6 or marker == 15 or marker == 3 or marker == 11 or marker == 13 or marker == 14 or marker == 4").query(
        "ecke == 0 or ecke == 3")

    t_boden = bodenpunkte.mean().values
    vec2 = hoeherepunkte.mean() - t_boden
    laenge = np.linalg.norm(vec2)
    vec2 /= laenge
    b = vec2.values
    faktor = 3.4 / laenge

    a = np.array([0, 0, 1])

    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    coords[['x', 'y', 'z']] -= t_boden
    coords[['x', 'y', 'z']] *= faktor
    coords[['x', 'y', 'z']] = coords[['x', 'y', 'z']].dot(rotation_matrix)

    for pict in pictures.values:
        t = pict[0:3].T
        R_org = cv2.Rodrigues(pict[3:])[0]
        R = np.linalg.inv(R_org)
        t = -R@t
        R_neu = np.linalg.inv(rotation_matrix@R_org)
        pict[0:3] = -R_neu@(((t-t_boden)*faktor)@rotation_matrix).T
        pict[3:] = cv2.Rodrigues(R_neu)[0].T
    return coords, pictures

In [102]:
def vergleich(imgs, coords, pictures, kameras):
    d = []
    for nr in pictures.index:
        r = pictures.query("img == "+str(nr))[["r1", "r2", "r3"]].values
        t = pictures.query("img == "+str(nr))[["x", "y", "z"]].values
        kp = kameras.query("kamera == 1").values
        mtx = np.array([[kp[0][0], 0, kp[0][2]], [
            0, kp[0][1], kp[0][3]], [0, 0, 1]])
        dist = kp[0][4:]
        img_points, _ = cv2.projectPoints(coords.values, r, t, mtx, dist)
        img_points = pd.DataFrame(
            img_points.reshape(-1, 2), columns=["x_neu", "y_neu"])
        img_points['img'] = nr
        img_points[["marker", "ecke"]] = coords.reset_index()[
            ["marker", "ecke"]]
        img_points.set_index(["marker", "ecke", "img"], inplace=True)
        j = img_points.join(corners, lsuffix="_neu", rsuffix="_alt").dropna()
        j['dx'] = j['x_neu'] - j['x']
        j['dy'] = j['y_neu'] - j['y']
        j['d'] = np.sqrt(j['dx']**2 + j['dy']**2)

        if (j[['d']].std().values[0] > 40):
            print("Entferne Bild ", nr+1)
            pictures.drop(nr, inplace=True)
        elif j[j['d'] > 15].count().values[0] > 1:
            for i in j[j['d'] > 15].index:
                print("Entferne Passpunkt ", i)
                print(i)
                coords.drop(i[:2], inplace=True)


# vergleich(imgs, coords, pictures, kameras)

### Ausgleichung

In [129]:
def passpunkt_groesse(coords):
    l = []
    A = []
    m = 3.4
    for marker in coords.reset_index()['marker'].unique():
        #print(marker)
        if marker == 0:
            m = 15.6
        elif marker >= 24:
            m = 7.2
        else:
            m = 3.4
        m1 = np.where(coords.index == (marker, 0))[0][0]
        m2 = np.where(coords.index == (marker, 1))[0][0]
        m3 = np.where(coords.index == (marker, 2))[0][0]
        m4 = np.where(coords.index == (marker, 3))[0][0]
        l.append([m, m*2**0.5, m, m, m*2**0.5, m])
        A.append([m1*2, m1*2 + 1, m2*2, m2*2+1])
        A.append([m1*2, m1*2 + 1, m3*2, m3*2+1])
        A.append([m1*2, m1*2 + 1, m4*2, m4*2+1])
        A.append([m2*2, m2*2 + 1, m3*2, m3*2+1])
        A.append([m2*2, m2*2 + 1, m4*2, m4*2+1])
        A.append([m3*2, m3*2 + 1, m4*2, m4*2+1])
    l = np.array(l)
    #print(l)
    A = np.array(A)
    return A, l.ravel()


In [137]:
coords.dropna(inplace=True)
pictures.dropna(inplace=True)

In [187]:
def ausgleichung(coords, pictures, kameratypen, kameras, corners):
    lp = corners.join(coords, lsuffix="_img", rsuffix="").join(pictures, lsuffix="", rsuffix="_cam").dropna()

    Apg, lpg = passpunkt_groesse(coords)

    kameratyp = kameratypen.reset_index().values
    kameratyp_ids = np.array(kameratyp[:, 0], dtype=np.int32)

    kamera = kameras.reset_index().values
    kamera_ids = np.array(kamera[:, :2], dtype=np.int32)

    bilder = pictures.reset_index().dropna().drop(
        'pfad', axis=1).to_numpy(dtype=np.float32)
    bilder_ids = np.array(bilder[:, :2], dtype=np.int32)

    passpunkte = coords.reset_index().values
    passpunkte_ids = np.array(passpunkte[:, :2], dtype=np.int32)

    messung = lp.reset_index().to_numpy()[:, :5]
    messung_ids = np.array(messung[:, :3], dtype=np.int32)

    l = np.hstack(
        (messung[:, -2:].astype(np.float32).ravel(), lpg.ravel()), dtype=np.float32)

    x0 = np.hstack((kameratyp[:, 2:].astype(np.float32).ravel(),
                    kamera[:, 3:].astype(np.float32).ravel(),
                    bilder[:, 2:].astype(np.float32).ravel(),
                    passpunkte[:, 2:].astype(np.float32).ravel()), dtype=np.float32)

    A = lil_matrix((len(l), len(x0)), dtype=int)

    n_cameratypes = len(kameratyp)
    n_camera = len(kamera)
    n_bilder = len(bilder)
    n_passpunkte = len(passpunkte)
    n_messungen = len(messung)
    messung_bild_id = np.empty(n_messungen, dtype=np.int32)
    messung_kamera_id = np.empty(n_messungen, dtype=np.int32)
    messung_kameratypen_id = np.empty(n_messungen, dtype=np.int32)
    messung_passpunkt_id = np.empty(n_messungen, dtype=np.int32)
    num_cam_type_param = 8
    num_cam_param = 3
    num_bild_param = 6
    num_pass_param = 3

    offset = n_cameratypes * num_cam_type_param + n_camera*num_cam_param + n_bilder * num_bild_param

    for i, a in enumerate(Apg):
        zeile = n_messungen + i
        for j in a:
            A[zeile, offset+j] = 1

    for i, m in enumerate(messung):
        #print(m)
        bild_id, = np.where(bilder_ids[:, 0] == m[2])
        bild_id = bild_id[0]
        messung_bild_id[i] = bild_id

        camera_id_array,  = np.where(kamera_ids[:,0] == bilder[bild_id, 1])
        camera_id = camera_id_array[0]
        messung_kamera_id[i] = camera_id

        cameratype_id_array, = np.where(kameratyp_ids[:] == kamera[camera_id, 1])
        cameratype_id = cameratype_id_array[0]
        messung_kameratypen_id[i] = cameratype_id

        passpunkt_id_array, = np.where(
            (passpunkte_ids[:, 0] == m[0]) & (passpunkte_ids[:, 1] == m[1]))
        passpunkt_id = passpunkt_id_array[0]
        messung_passpunkt_id[i] = passpunkt_id

        offset = cameratype_id * num_cam_type_param
        A[2*i:2*i+2, offset:offset+num_cam_type_param] = 1

        offset = n_cameratypes * num_cam_type_param + camera_id*num_cam_param
        A[2*i:2*i+2, offset:offset+num_cam_param] = 1

        offset = n_cameratypes * num_cam_type_param + \
            n_camera * num_cam_param + bild_id * num_bild_param
        A[2*i:2*i+2, offset:offset + num_bild_param] = 1

        offset = n_cameratypes * num_cam_type_param + n_camera*num_cam_param + n_bilder * \
            num_bild_param + passpunkt_id * num_pass_param
        A[2*i:2*i+2, offset:offset + num_pass_param] = 1

    def project(x0: NDArray[np.float32]) -> NDArray[np.float32]:
        #print(x0)
        p = np.empty(len(l), dtype=np.float32)

        K = []
        dist = []
        for i in range(n_camera):
            offset = num_cam_type_param * n_cameratypes + i * num_cam_param
            typ_id, = np.where(kameratyp_ids[:] == kamera_ids[i, 1])
            typ_id = typ_id[0]
            typoffset = num_cam_type_param * typ_id
            f = x0[typoffset] + x0[offset]
            cx = x0[typoffset+1] + x0[offset + 1]
            cy = x0[typoffset+2] + x0[offset + 2]
            K.append(np.array([[f, 0, cx], [0, f, cy], [0, 0, 1]]))
            dist.append(np.array([x0[typoffset + 3], x0[typoffset + 4],
                                  x0[typoffset + 5], x0[typoffset + 6], x0[typoffset + 7]]))
        #print(K)
        r = []
        t = []
        for i in range(n_bilder):
            offset = num_cam_type_param * n_cameratypes + n_camera * num_cam_param + \
                i*num_bild_param

            t.append(x0[offset:offset+3])
            r.append(x0[offset+3:offset+6])

        X = []
        for i in range(n_passpunkte):
            offset = num_cam_type_param * n_cameratypes + n_camera * num_cam_param + \
                n_bilder*num_bild_param + \
                num_pass_param * i
            X.append(x0[offset:offset+3].reshape(1, 3))

        for i, m in enumerate(messung):
            punkt, _ = cv2.projectPoints(
                X[messung_passpunkt_id[i]], r[messung_bild_id[i]], t[messung_bild_id[i]], K[messung_kamera_id[i]], dist[messung_kamera_id[i]])
            #if (i < 8): print(punkt)
            p[i*2] = punkt[0, 0, 0]
            p[i*2+1] = punkt[0, 0, 1]

        offset = n_messungen * 2
        for i, marker in enumerate(coords.reset_index()['marker'].unique()):
            #print(marker)
            if marker == 0:
                m = 15.6
            else:
                m = 3.4
            m1 = np.where(coords.index == (marker, 0))[0][0]
            m2 = np.where(coords.index == (marker, 1))[0][0]
            m3 = np.where(coords.index == (marker, 2))[0][0]
            m4 = np.where(coords.index == (marker, 3))[0][0]
            
            p[offset + i+0] = np.linalg.norm(coords.values[m1] - coords.values[m2])
            p[offset + i +
                1] = np.linalg.norm(coords.values[m1] - coords.values[m3])
            p[offset + i +
                2] = np.linalg.norm(coords.values[m1] - coords.values[m4])
            p[offset + i +
                3] = np.linalg.norm(coords.values[m2] - coords.values[m3])
            p[offset + i +
                4] = np.linalg.norm(coords.values[m2] - coords.values[m4])
            p[offset + i +
                5] = np.linalg.norm(coords.values[m3] - coords.values[m4])
            #print(p[i:i+6])

        #print(np.c_[l[:15],p.ravel()[:15]])
        return np.array(l-p.ravel(), dtype=np.float32)
    #print(x0)
    #print(l)
    res = least_squares(project, x0, jac_sparsity=A, verbose=2,
                        x_scale='jac', method='trf')  # type: ignore

    #print()
    x = res.x

    kameratypen = pd.DataFrame(np.c_[kameratyp_ids, x[:num_cam_type_param * n_cameratypes].reshape(n_cameratypes, num_cam_type_param)],
                           columns=["kameratyp", "f", "cx", "cy", "d1", "d2", "d3", "d4", "d5"]).astype({"kameratyp": int}).set_index("kameratyp")

    kameras = pd.DataFrame(np.c_[kamera_ids, x[num_cam_type_param * n_cameratypes:num_cam_type_param * n_cameratypes + num_cam_param * n_camera].reshape(n_camera, num_cam_param)],
                           columns=["kamera", "kameratyp", "df", "dcx", "dcy"]).astype({"kamera": int, "kameratyp": int}).set_index("kamera")

    offset = num_cam_type_param * n_cameratypes + num_cam_param * n_camera
    #print(bilder_ids[:, 1])
    neue_pictures = pd.DataFrame(np.c_[bilder_ids[:, 0], x[offset:offset+num_bild_param*n_bilder].reshape(
        n_bilder, num_bild_param)], columns=["img", "x", "y", "z", "r1", "r2", "r3"]).astype({"img": int}).set_index("img")
    #print(neue_pictures)
    pictures.update(neue_pictures)

    offset = num_cam_param * n_camera + num_bild_param*n_bilder
    coords = pd.DataFrame(np.c_[passpunkte_ids, x[offset:offset+num_pass_param*n_passpunkte].reshape(n_passpunkte, num_pass_param)], columns=["marker", "ecke", "x", "y", "z"]).astype({"marker": int, "ecke": int}).set_index(["marker", "ecke"])

    # print(project(x).reshape(-1, 2))
    return res, kameratypen, kameras, pictures, coords,l,x0,project(x0)

In [127]:
vorher_coords = coords.copy()
vorher_pictures = pictures.copy()
vorher_kameras = kameras.copy()

In [189]:
res, kameratypen, kameras, pictures, coords, l, x0, px0 = ausgleichung(
    coords, pictures, kameratypen, kameras, corners)

ValueError: cannot reshape array of size 1 into shape (1,3)

#### Kontrolle

In [190]:
wolke_zeigen(coords, pictures)

In [191]:
kameratypen

,f,cx,cy,d1,d2,d3,d4,d5
kameratyp,,,,,,,,
0,3386.320077,2319.903602,1297.24586,0.120319,-0.16663,0.034366,-0.071421,-0.44754


In [192]:
kameras

,kameratyp,df,dcx,dcy
kamera,,,,
0,0,75.623625,1.228496,-20.512582


In [193]:
pictures

,kamera,pfad,t1,t2,t3,r1,r2,r3
img,,,,,,,,
0,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
1,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
2,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,-0.272067,2.767619,13.904568,0.888809,-1.934053,1.502359
3,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
4,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
5,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
6,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
7,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN
8,0,../../bilderserien/33a7d422-5c80-4444-826e-375...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def zeige_auf_bildern(coords, pictures, kameras):
    picWC = pictures.dropna().copy()
    picWC[['x', 'y', 'z']] = [-(cv2.Rodrigues(np.array(list(p[['r1', 'r2', 'r3']].values)))[0]).T @ p[['t1','t2','t3']].values.T for i,p in pictures.dropna().iterrows()]

    for nr in picWC.index:
        print(nr)
        imgCV = cv2.imread(pictures.query("img == "+str(nr))['pfad'].values[0])
        r = pictures.query("img == "+str(nr))[["r1", "r2", "r3"]].to_numpy(dtype=np.float64)[0]
        t = pictures.query("img == "+str(nr))[["t1", "t2", "t3"]].to_numpy(dtype=np.float64)
        kamera = pictures.query("img == "+str(nr))['kamera'].values[0]
        print(kamera)
        kp = kameras.query("kamera == "+ str(kamera)).values
        mtx = np.array([[kp[0][0], 0, kp[0][2]], [
            0, kp[0][1], kp[0][3]], [0, 0, 1]])
        dist = kp[0][4:]
        for p in corners.query("img == "+str(nr)).reset_index().values:
            cv2.circle(imgCV, (int(p[3]), int(p[4])), 10, (255, 0, 0), -1)
            cv2.putText(imgCV, str(int(p[1]))+'-'+str(int(p[2])), (int(p[3]),
                        int(p[4])),  cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 5)
        img_points, _ = cv2.projectPoints(coords.values, r, t, mtx, dist)
        for c, p in zip(coords.reset_index()['marker'].values, img_points):
            try:
                cv2.circle(imgCV, (int(p[0][0]), int(
                    p[0][1])), 10, (0, 0, 255), -1)
                cv2.putText(imgCV, str(c), (int(p[0][0]), int(
                    p[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)
            except:
                pass
        
        cam_points, _ = cv2.projectPoints(
            picWC[['x', 'y', 'z']].values, r, t, mtx, dist)
        for c, p in zip(picWC.reset_index()['img'].values, cam_points):
            if nr == c:
                continue
            try:
                cv2.circle(imgCV, (int(p[0][0]), int(
                    p[0][1])), 10, (0, 255, 0), -1)
                cv2.putText(imgCV, str(c+1), (int(p[0][0]), int(
                    p[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 2)
            except:
                pass
        cv2.drawFrameAxes(imgCV, mtx, dist, r, t, 10,10)
        plt.figure(figsize=(14, 11))
        plt.imshow(imgCV)
        plt.show()


zeige_auf_bildern(coords, pictures, kameras)

In [ ]:
picWC = pictures.dropna().copy()
picWC[['x', 'y', 'z']] = [-np.linalg.inv(cv2.Rodrigues(np.array(list(p[['r1', 'r2', 'r3']].values)))[0]) @ p[['t1','t2','t3']].values.T for i,p in pictures.dropna().iterrows()]
picWC

In [ ]:
pictures.loc[3,['t1', 't2', 't3', 'r1', 'r2', 'r3']] = [None, None, None,None,None,None]

In [ ]:
res.jac.todense().shape


In [ ]:
from scipy.sparse import linalg, diags


cov = linalg.inv(res.jac.T @ res.jac)
test = np.sqrt(np.diag(cov.todense()))
test

In [ ]:
#coords.to_csv("pb_coords.csv")
#pictures.to_csv("pb_pictures.csv")
#kameras.to_csv("pb_kameras.csv")